# Scheme A: BatchedInferencePipeline Interactive Lab

This notebook demonstrates how to use the `BatchedInferencePipeline` for faster transcription and VAD.

In [ ]:
import time
import torch
from faster_whisper import WhisperModel, BatchedInferencePipeline

# 检查显卡
print(f"GPU Available: {torch.cuda.is_available()}")
print(f"Device Name: {torch.cuda.get_device_name(0)}")

# 1. 加载模型 (这一步大概需要 10-20秒下载模型)
print("🔄 Loading Whisper Model (Large-v3)...")
model = WhisperModel("large-v3", device="cuda", compute_type="float16")

# 2. 封装 Batch Pipeline (方案A的核心)
batched_model = BatchedInferencePipeline(model=model)

print("✅ Model Loaded & Pipeline Ready!")

In [ ]:
import os

# 示例：下载一段音频 (如果你通过 Jupyter 的上传按钮上传了文件，跳过这行)
test_url = "https://r2.deth.us/test/20251212120025488-b72a3e5a-podcast.mp3"
audio_file = "test_audio.mp3"

if not os.path.exists(audio_file):
    print(f"⬇️ Downloading sample audio from {test_url}...")
    os.system(f"curl -L -o {audio_file} {test_url}")
    
print(f"📂 Target Audio: {audio_file}")

In [ ]:
print("🔥 Starting Batched Inference...")
start_time = time.time()

# 核心推理
segments_generator, info = batched_model.transcribe(
    audio_file, 
    batch_size=16, 
    language="zh"
)

segments = list(segments_generator)

end_time = time.time()
duration = end_time - start_time

print(f"✅ Finished!")
print(f"⏱️ Wall Time: {duration:.2f}s")
print(f"📊 Audio Duration: {info.duration:.2f}s")
print(f"⚡ Speed Factor: {info.duration / duration:.1f}x real-time")

In [ ]:
import pandas as pd

data = []
for seg in segments:
    data.append({
        "start": seg.start,
        "end": seg.end,
        "text": seg.text,
        "confidence": seg.avg_logprob
    })

df = pd.DataFrame(data)
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_colwidth', 100)

display(df.head(10))

df.to_csv("result.csv", index=False)
print("💾 Saved to result.csv")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

fig, ax = plt.subplots(figsize=(20, 2))

for seg in segments:
    width = seg.end - seg.start
    rect = patches.Rectangle((seg.start, 0), width, 1, linewidth=1, edgecolor='black', facecolor='lightgreen')
    ax.add_patch(rect)

ax.set_xlim(0, info.duration)
ax.set_ylim(0, 1)
ax.set_xlabel("Time (seconds)")
ax.set_yticks([])
ax.set_title("VAD Segmentation & Transcription Timeline (Scheme A)")

plt.show()